# 🐺 TEST 3: What's Actually Moving RIGHT NOW?
## Find the REAL setups for Monday - not theory, REALITY

**Goal:** Scan candidates, find what to actually buy Monday

**Decision Rule:**
- RSI < 65 + momentum + pullback = BUY
- RSI > 70 = WAIT
- No clear setup = DON'T TRADE

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Ready to scan")

---
## Your Watchlist

In [ ]:
# Your current positions
POSITIONS = {
    'APLD': 4,   # shares owned
    'KTOS': 1,
    'TLRY': 10,
    'NTLA': 10
}

# Potential entries
CANDIDATES = [
    'SOUN', 'BBAI', 'RCAT', 'OUST',  # Voice AI, Defense, LiDAR
    'CIFR', 'IREN', 'WULF', 'CORZ',  # AI Infra
    'SMR', 'NNE', 'OKLO', 'VST',     # Nuclear
    'RKLB', 'LUNR', 'RDW',           # Space
    'IONQ', 'RGTI',                  # Quantum
    'DNN', 'UEC', 'UUUU'             # Uranium
]

print(f"📊 Positions: {len(POSITIONS)}")
print(f"🎯 Candidates: {len(CANDIDATES)}")

---
## Scan Function

In [ ]:
def calculate_rsi(prices, period=14):
    """Calculate RSI"""
    deltas = np.diff(prices)
    seed = deltas[:period+1]
    up = seed[seed >= 0].sum() / period
    down = -seed[seed < 0].sum() / period
    rs = up / down if down != 0 else 0
    rsi = np.zeros_like(prices)
    rsi[:period] = 100. - 100. / (1. + rs)

    for i in range(period, len(prices)):
        delta = deltas[i - 1]
        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up * (period - 1) + upval) / period
        down = (down * (period - 1) + downval) / period

        rs = up / down if down != 0 else 0
        rsi[i] = 100. - 100. / (1. + rs)

    return rsi[-1]


def scan_ticker(ticker):
    """Scan a ticker for entry setup"""
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period='1mo')
        info = stock.info
        
        if len(hist) < 15:
            return None
        
        # Current data
        current = hist['Close'].iloc[-1]
        week_ago = hist['Close'].iloc[-5] if len(hist) >= 5 else hist['Close'].iloc[0]
        week_chg = ((current - week_ago) / week_ago) * 100
        
        # RSI
        rsi = calculate_rsi(hist['Close'].values)
        
        # Volume
        vol = hist['Volume'].iloc[-1]
        avg_vol = hist['Volume'].mean()
        vol_ratio = vol / avg_vol if avg_vol > 0 else 1
        
        # 52 week position
        high_52 = info.get('fiftyTwoWeekHigh', current)
        from_high = ((high_52 - current) / high_52) * 100
        
        # Entry assessment
        if rsi > 70:
            entry = "WAIT"
            reason = f"RSI {rsi:.0f} overbought"
        elif rsi < 30:
            entry = "WATCH"
            reason = f"RSI {rsi:.0f} oversold - could bounce or continue"
        elif 20 <= from_high <= 50 and week_chg > 0 and rsi < 65:
            entry = "BUY"
            reason = "Pullback + momentum + RSI good"
        elif week_chg > 5 and vol_ratio > 1.2 and rsi < 65:
            entry = "BUY"
            reason = "Momentum building"
        elif week_chg < -15 and rsi < 40:
            entry = "BUY"
            reason = "Crash bounce setup"
        else:
            entry = "SKIP"
            reason = "No clear setup"
        
        return {
            'ticker': ticker,
            'price': current,
            'week_chg': week_chg,
            'rsi': rsi,
            'vol_ratio': vol_ratio,
            'from_high': from_high,
            'entry': entry,
            'reason': reason
        }
    except Exception as e:
        return None

print("✅ Scan function ready")

---
## Check Your Positions

In [ ]:
print("\n" + "="*70)
print("📊 YOUR CURRENT POSITIONS")
print("="*70)

for ticker, shares in POSITIONS.items():
    result = scan_ticker(ticker)
    if result:
        print(f"\n{ticker} - {shares} shares @ ${result['price']:.2f}")
        print(f"   Week: {result['week_chg']:+.1f}% | RSI: {result['rsi']:.0f} | Vol: {result['vol_ratio']:.1f}x")
        print(f"   Status: {result['entry']} - {result['reason']}")

---
## Scan All Candidates

In [ ]:
print("\n" + "="*70)
print("🔍 SCANNING CANDIDATES...")
print("="*70)

results = []

for ticker in CANDIDATES:
    result = scan_ticker(ticker)
    if result:
        results.append(result)

# Sort by entry priority
buy_now = [r for r in results if r['entry'] == 'BUY']
watch_list = [r for r in results if r['entry'] == 'WATCH']
wait_list = [r for r in results if r['entry'] == 'WAIT']

# Display BUY NOW
if buy_now:
    print("\n🟢 BUY NOW - Good Setups:")
    print("-" * 70)
    
    for r in sorted(buy_now, key=lambda x: x['week_chg'], reverse=True):
        shares = int(100 / r['price'])  # ~$100 position
        print(f"\n  {r['ticker']} @ ${r['price']:.2f}")
        print(f"     Week: {r['week_chg']:+.1f}% | RSI: {r['rsi']:.0f} | Vol: {r['vol_ratio']:.1f}x | -{r['from_high']:.1f}% from high")
        print(f"     ✅ {r['reason']}")
        print(f"     Position: {shares} shares (~${shares * r['price']:.0f})")

if watch_list:
    print("\n🟡 WATCH - Potential Bounces:")
    print("-" * 70)
    
    for r in watch_list:
        print(f"  {r['ticker']} @ ${r['price']:.2f} - {r['reason']}")

if wait_list:
    print("\n🔴 WAIT - Overbought:")
    print("-" * 70)
    
    for r in wait_list[:5]:  # Show top 5
        wait_price = r['price'] * 0.93  # Wait for 7% pullback
        print(f"  {r['ticker']} @ ${r['price']:.2f} - RSI {r['rsi']:.0f} (wait for ${wait_price:.2f})")

# Summary
print("\n" + "="*70)
print(f"📊 SUMMARY: {len(buy_now)} BUY | {len(watch_list)} WATCH | {len(wait_list)} WAIT")

if len(buy_now) > 0:
    print("\n✅ ACTION: Execute BUY NOW list Monday 10 AM")
else:
    print("\n❌ ACTION: NO TRADES - Wait for better setups")
print("="*70)

---
## 🎯 MONDAY PLAN

**Copy-paste this into your trading notes:**

In [ ]:
if len(buy_now) > 0:
    print("\n" + "="*70)
    print("📋 MONDAY JANUARY 13 TRADE PLAN")
    print("="*70)
    
    total_capital = 420  # After TLRY sale
    max_positions = min(3, len(buy_now))
    position_size = total_capital / max_positions
    
    print(f"\nCapital: ${total_capital}")
    print(f"Max Positions: {max_positions}")
    print(f"Position Size: ${position_size:.0f} each")
    
    print(f"\n🎯 EXECUTION LIST:")
    
    for i, r in enumerate(buy_now[:max_positions], 1):
        shares = int(position_size / r['price'])
        cost = shares * r['price']
        stop = r['price'] * 0.95
        target = r['price'] * 1.08
        
        print(f"\n{i}. {r['ticker']}")
        print(f"   Entry: ${r['price']:.2f} x {shares} shares = ${cost:.2f}")
        print(f"   Stop: ${stop:.2f} (-5%)")
        print(f"   Target: ${target:.2f} (+8%)")
        print(f"   Reason: {r['reason']}")
    
    print(f"\n⏰ TIMING:")
    print(f"   8:30 AM - Check CPI data")
    print(f"   9:30 AM - Sell TLRY (${92})")
    print(f"   10:00 AM - Execute entries above")
    print(f"   10:05 AM - Set stop losses")
    
    print("\n" + "="*70)
else:
    print("\n📋 MONDAY PLAN: NO TRADES")
    print("   No clear setups. Wait for better entries.")

---
## 💡 THIS IS THE WORKFLOW

Run this notebook every weekend:
- See what's actually moving
- Get clear BUY/WAIT signals
- Copy Monday plan
- Execute

**No guessing. Data decides.**